In [1]:
!pip install git+https://github.com/ggumtakg/py-hanspell
!pip install konlpy

  Cloning https://github.com/ggumtakg/py-hanspell to /tmp/pip-req-build-tjumjo56
  Running command git clone --filter=blob:none --quiet https://github.com/ggumtakg/py-hanspell /tmp/pip-req-build-tjumjo56
  Resolved https://github.com/ggumtakg/py-hanspell to commit 3bc19f0cfc61d158afc46a250aad4230eaad6435
  Preparing metadata (setup.py) ... done


# EDA & 전처리
- 데이터 샘플 및 기초정보 확인
- 특수문자 삭제 및 영문 소문자화
- /n을 띄워쓰기로 변환
- 욕설 필터링
- KoNLPy의 형태소 분석기(mecab)를 통한 토크나이징 후
  - 텍스트 길이 분포 확인
  - 단어 빈도 분석
  - 불용어 처리 -> 불용어는 형태소분석기를 통한 토크나이징 후 해당 불용어만 존재하는 토큰을 삭제
- 결측값 제거
- 중복데이터 제거
- 단어 임베딩
- 패딩을 통한 길이 통일
- 원핫 인코딩을 통한 클래스 인코딩

In [2]:
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

from konlpy.tag import Mecab  # 형태소 분석기
from collections import Counter # 빈도 분석
from wordcloud import WordCloud # 빈도 시각화
from gensim.models import Word2Vec  # 단어 임베딩
from sklearn.model_selection import train_test_split  # train, validation분리
from hanspell import spell_checker  # 맞춤법 검사기

### 데이터 로드 및 기초정보 확인

In [3]:
# 데이터 로드
train_data = pd.read_csv('/workspace/train.csv')

# 데이터 샘플 확인
print(train_data.head())

# 기본 정보 확인 - null유무 및 데이터타입, 갯수 확인
print(train_data.info())

   idx  class                                       conversation
0    0  일반 대화  너희 오토바이 타봤어? 이번에 친구가 오토바이 사서 자랑하는데 너무 부럽더라 나는 ...
1    1  일반 대화  나 연수좀 도와줘 운전 연수 말하는거야? 하...또 이 베스트드라이버가 나설 차레인...
2    2  일반 대화  나 이번 주에 할머니 집 가. 그래? 오랜만에 할머니도 보고 좋겠네. 그러게, 할머...
3    3  일반 대화  학교 등교 할 때 택시타고 등교 하는데 차가 너무 많이 막혀 맞아, 요즘 차가 너무...
4    4  일반 대화  오늘 등굣길에 버스를 탔는데 사람이 너무 많았어. 학생들 등교 시간이라 사람이 많은...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4950 entries, 0 to 4949
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           4950 non-null   int64 
 1   class         4950 non-null   object
 2   conversation  4950 non-null   object
dtypes: int64(1), object(2)
memory usage: 116.1+ KB
None


### 텍스트 정규화
- 문장 양극단 공백 제거
- 과도하게 반복되는 문자 제거(ex. 키키키키키키 -> 키키)
- 불용 특수문자 삭제
- 영문 소문자화
- 여러개의 공백 하나로 치환(ex. 아니   뭐래 -> 아니 뭐래)
- 줄바꿈 문자 띄워쓰기로 치환

In [ ]:
# 맞춤법 검사기 초기화
spell_checker.setParam('8940e67396b6ae4844dfea8112e130c6e2d0e2f5', '1733453547941')

# 특수문자 삭제 및 영문 소문자화 함수
def preprocess_text(text):
    # 입력받은 문장의 양쪽 공백을 제거
    text = text.strip()

    # 반복 문자 제거
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)

    # 특수문자 삭제
    text = re.sub(r'[^가-힣a-zA-Z0-9?\s]', '', text)

    # 영문 소문자화
    text = text.lower()

    # 여러 개의 공백을 하나의 공백으로 변환
    text = re.sub(r'\s+', ' ', text)

    # 입력받은 문장의 양쪽 공백을 제거
    text = text.strip()

    # 줄바꿈문자를 띄워쓰기로 변환
    text.replace('\n', ' ')

    # 한글 맞춤법 교정
    corrected_sentences = []
    for sentence in text:
        corrected_sentence = spell_checker.check(sentence).checked
        corrected_sentences.append(corrected_sentence)
    return corrected_sentences

# 텍스트 전처리
train_data['conversation'] = train_data['conversation'].apply(preprocess_text)

In [ ]:
print(train_data.head())

### 여기까지 진행된 데이터 저장 및 이후로는 저장된 데이터 활용
- 맞춤법 검사기로 인한 실행시간이 길어짐

In [ ]:
train_data.to_csv('train_preproc.csv', index=False, encoding='utf-8')

train_data = pd.read_csv('train_preproc.csv')

### 욕설 필터링
- 욕설 단어만 제거
- 욕설 목록은 https://namu.wiki/w/%EC%9A%95%EC%84%A4/%ED%95%9C%EA%B5%AD%EC%96%B4를 참고하여 일부 발췌 했습니다.

In [ ]:
'''사용 여부에 따라 주석 해제 요망
# 욕설 리스트
bad_words = ['씨발', '개새끼', '니미']

# 욕설 제거 함수
def remove_bad_words(text):
    for bad_word in bad_words:
        text = text.replace(bad_word, '')
    return text

# 데이터 로드
train_data = pd.read_csv('train.csv')

# 욕설 제거
train_data['text'] = train_data['text'].apply(remove_bad_words)
'''

### 형태소 분석기
- KoNLPy의 Mecab사용

In [ ]:
# 형태소 분석기 초기화
mecab = Mecab()

# 형태소 분석 함수
def tokenize_text(text):
    return mecab.morphs(text)

# 형태소 분석
train_data['tokens'] = train_data['conversation'].apply(tokenize_text)

### 불용어 처리

In [ ]:
# 불용어 리스트
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

# 불용어 제거 함수
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopwords]

# 불용어 제거
train_data['tokens'] = train_data['tokens'].apply(remove_stopwords)

### 텍스트 길이 분포 확인

In [ ]:
# 텍스트 길이 분포 확인
train_data['text_length'] = train_data['tokens'].apply(len)

sns.histplot(train_data['text_length'], bins=50, kde=True)
plt.title('Text Length Distribution in Training Data')
plt.show()

### 단어 빈도 분석
- wordcloud를 통한 시각화

In [ ]:
# 단어 빈도 분석
train_words = [word for tokens in train_data['tokens'] for word in tokens]
train_word_freq = Counter(train_words)

# wordcloud 시각화
wordcloud = WordCloud(width=800, height=400, background_color='white', font_path='path/to/your/font.ttf').generate_from_frequencies(train_word_freq)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud for Training Data')
plt.show()

### 결측치 및 중복 데이터 삭제

In [ ]:
# 결측값 제거
train_data.dropna(inplace=True)

# 중복 데이터 제거
train_data['conversation'].drop_duplicates(inplace=True)

### 단어 임베딩
- word2vec 사용

In [ ]:
# Word2Vec 모델 학습
sentences = train_data['tokens'].tolist()
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# 단어 임베딩 벡터 생성
def get_word_embedding(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return [0] * model.vector_size

# 단어 임베딩 벡터 생성
train_data['embedding'] = train_data['tokens'].apply(lambda x: get_word_embedding(x, word2vec_model))

### pad_sequences를 통한 패딩 추가

In [ ]:
# 패딩 함수
MAX_LENGTH = 100
def pad_sequences(sequences, maxlen=MAX_LENGTH, padding='post', truncating='post'):
    padded_sequences = []
    for seq in sequences:
        if len(seq) > maxlen:
            seq = seq[:maxlen]
        else:
            seq = seq + [0] * (maxlen - len(seq))
        padded_sequences.append(seq)
    return np.array(padded_sequences)

# 패딩 적용
train_embeddings = pad_sequences(train_data['embedding'].tolist(), maxlen=MAX_LENGTH)

### 클래스 원 핫 인코딩

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# 원핫 인코딩
encoder = OneHotEncoder(sparse=False)
train_labels = encoder.fit_transform(train_data[['class']])

### train, validation 데이터셋 분리
- 비율 0.2

In [ ]:
# 데이터셋 분할
X_train, X_val, y_train, y_val = train_test_split(train_embeddings, train_labels, test_size=0.2, random_state=42)

# 데이터셋 준비
train_dataset = (X_train, y_train)
val_dataset = (X_val, y_val)

### 결과 저장

In [ ]:
train_dataset.to_csv('train_dataset.csv', index=False, encoding='utf-8')
val_dataset.to_csv('val_dataset.csv', index=False, encoding='utf-8')